In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [2]:
# Load Train, Validation, Test Dataset
dataset = pd.read_csv('/kaggle/input/mimic-bengali-preprocessed/balanced_translated_dataset_preprocessed.csv')

In [3]:
dataset.head()

,english_text,english_text_preprocessed,label,bengali_text
0,CHIEF COMPLAINT: weak/fatigue\n\nPRESENT ILLNE...,chief complaint weakfatigue present illness ye...,0,"প্রধান অভিযোগ হল, দুর্দশাগ্রস্ত রোগ বর্তমান পু..."
1,CHIEF COMPLAINT: Fever and hypotension.\n\nPRE...,chief complaint fever hypotension present illn...,0,প্রধান অভিযোগ জ্বর হাইপোটেনশন বর্তমান অসুস্থ র...
2,CHIEF COMPLAINT: fall\n\nPRESENT ILLNESS: HPI:...,chief complaint fall present illness hpi f fal...,0,"প্রধান অভিযোগ হচ্ছে, গত রাতে হেডনো লক্রেট্রেশন..."
3,CHIEF COMPLAINT: Worsening shortness of breath...,chief complaint worsening shortness breath pre...,0,"প্রধান অভিযোগ হচ্ছে, রোগী রোগী বছর বয়সী মহিলা..."
4,CHIEF COMPLAINT: Chest heaviness\n\nPRESENT IL...,chief complaint chest heaviness present illnes...,0,প্রধান অভিযোগের বুকটি বর্তমান অসুস্থতাকে ক্ষুব...


In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train (80%) and temp (20%)
train_df, temp_df = train_test_split(dataset, test_size=0.2, stratify=dataset['label'], random_state=42)

# Split the temp data into validation (10%) and test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Rename the datasets
train_dataset = train_df
val_dataset = val_df
test_dataset = test_df

In [5]:
# Print class distribution for the train dataset
print("Train dataset class distribution:")
print(train_dataset['label'].value_counts())

# Print class distribution for the validation dataset
print("Validation dataset class distribution:")
print(val_dataset['label'].value_counts())

# Print class distribution for the test dataset
print("Test dataset class distribution:")
print(test_dataset['label'].value_counts())

Train dataset class distribution:
label
1    800
0    800
Name: count, dtype: int64
Validation dataset class distribution:
label
1    100
0    100
Name: count, dtype: int64
Test dataset class distribution:
label
1    100
0    100
Name: count, dtype: int64


In [6]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        output1 = self.model1(input_ids, attention_mask=attention_mask)[0]
        avg_output = output1
        return avg_output

In [7]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# Specify the dropout rate in the configuration
config = AutoConfig.from_pretrained('sagorsarker/bangla-bert-base', 
                                    num_labels=2, 
                                    hidden_dropout_prob=0.2, 
                                    attention_probs_dropout_prob=0.2)

# Load the pre-trained model with the specified configuration
bert_model = AutoModelForSequenceClassification.from_pretrained('sagorsarker/bangla-bert-base', config=config)


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

In [9]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['bengali_text'].tolist(), truncation=True, padding=True, max_length = 512)
val_encodings = tokenizer(val_dataset['bengali_text'].tolist(), truncation=True, padding=True,  max_length = 512)
test_encodings = tokenizer(test_dataset['bengali_text'].tolist(), truncation=True, padding=True , max_length = 512)

In [10]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = LosDataset(train_encodings, train_dataset['label'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['label'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['label'].tolist())

In [12]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(bert_model)

In [13]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=18, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=18, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=18, shuffle=False)

In [15]:
epochs = 200
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 10
gradient_accumulation_steps = 10
best_model_path = "best_model.pth"

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

best_model_state_dict = None

# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class)
    precision = precision_score(val_labels, val_preds_class)
    f1 = f1_score(val_labels, val_preds_class)
    roc_auc = roc_auc_score(val_labels, val_preds[:, 1])

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')

    # Implement early stopping
    if epoch > 0 and roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = roc_auc
        early_stopping_count = 0
        best_model_state_dict = ensemble_model.state_dict()
        torch.save(ensemble_model.state_dict(), f"BERT_baseline_MP_2000.pth")


100%|██████████| 12/12 [00:03<00:00,  3.83it/s]


Epoch: 1/200, Training Loss: 0.7076852877488297, Validation Loss: 0.7092338254054388
Accuracy: 0.475, Recall: 0.66, Precision: 0.48175182481751827, F1: 0.5569620253164558, Roc Auc: 0.48580000000000007


100%|██████████| 12/12 [00:03<00:00,  3.82it/s]


Epoch: 2/200, Training Loss: 0.703408987334605, Validation Loss: 0.7083308945099512
Accuracy: 0.49, Recall: 0.72, Precision: 0.4931506849315068, F1: 0.5853658536585366, Roc Auc: 0.4845
EarlyStopping counter: 1 out of 10


100%|██████████| 12/12 [00:03<00:00,  3.82it/s]


...

Epoch: 24/200, Training Loss: 0.18493987753819885, Validation Loss: 1.3440769215424855
Accuracy: 0.575, Recall: 0.85, Precision: 0.5483870967741935, F1: 0.6666666666666665, Roc Auc: 0.6178
EarlyStopping counter: 8 out of 10


100%|██████████| 12/12 [00:03<00:00,  3.81it/s]


Epoch: 25/200, Training Loss: 0.1872397368627318, Validation Loss: 1.224830001592636
Accuracy: 0.605, Recall: 0.6, Precision: 0.6060606060606061, F1: 0.6030150753768845, Roc Auc: 0.6283
EarlyStopping counter: 9 out of 10


100%|██████████| 12/12 [00:03<00:00,  3.81it/s]

Epoch: 26/200, Training Loss: 0.1695151004228699, Validation Loss: 1.3045382897059123
Accuracy: 0.605, Recall: 0.77, Precision: 0.5789473684210527, F1: 0.6609442060085837, Roc Auc: 0.6131
EarlyStopping counter: 10 out of 10
Early stopping


In [17]:
# Load the best model state dict into the model if needed
ensemble_model.load_state_dict(best_model_state_dict)

<All keys matched successfully>

In [18]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())


100%|██████████| 12/12 [00:03<00:00,  3.81it/s]


In [19]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class)
precision = precision_score(test_labels, test_preds_class)
f1 = f1_score(test_labels, test_preds_class)
roc_auc = roc_auc_score(test_labels, test_preds[:, 1])

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')

Accuracy: 0.555, Recall: 0.68, Precision: 0.544, F1: 0.6044444444444443, Roc Auc: 0.6106
